In [ ]:
import follow_the_leader


In [ ]:
import finesse
import finesse.analysis.actions as fac
import gpstime
import pickle
import argparse
import importlib
import finesse_ligo
from munch import Munch
#from gwpy.timeseries import TimeSeries
import numpy as np
import os
import json
from copy import deepcopy
from pathlib import Path
from finesse.exceptions import LostLock
#from gwpy.time import to_gps
from finesse_ligo.factory import ALIGOFactory

from follow_the_leader.actions import (
    add_fields_ligo,
    correct_arm_loss_for_nonzero_maxtem,
    MyLockLIGO,
    time_dependant_sim
)
from follow_the_leader.gwd_data import (
    llo_data
)

import sys

In [ ]:
LIGO_COMMISSIONING_DIR = follow_the_leader.add_power_up_scripts_to_path()
print(sys.path)

In [ ]:
import fenicsx; print(fenicsx.__file__) # should be called from the ligo-commissioning-modelling repo
import dmd; print(dmd.__file__)
import fdm; print(fdm.__file__)

In [ ]:
thermal_state_class = fenicsx.FENICSXThermalState
thermal_state_class = dmd.DMDThermalState # Comment this out to use Fenicxs
thermal_state_class = fdm.FDMThermalState # Comment this out to use Fenicxs

In [ ]:
factory = ALIGOFactory(LIGO_COMMISSIONING_DIR / "LLO" / "yaml" / "llo_O4.yaml")
# Make the ifo model
factory.options.LSC.add_output_detectors = True
factory.options.ASC.add = False
factory.options.thermal.add = True
llo = factory.make()
llo.modes("even", maxtem=6)
add_fields_ligo(llo)

In [ ]:
TS = thermal_state_class(factory, llo)
initial_model = llo.deepcopy()  # keep a copy of the initial state
TS.update_maps()

In [ ]:
correct_arm_loss_for_nonzero_maxtem(llo, verbose=True)

In [ ]:
lock_params = dict(
    exception_on_lock_fail=False,
    lock_steps=100,
    gain_scale=0.4,
    pseudo_lock_arms=False,
    run_locks=True,
    break_point=4
)


lock = MyLockLIGO(**lock_params)
sol = llo.run(lock)
sol["final run locks"].plot()

# save initial gains to be changed during power up
initial_gains = {}
for lsc_dof in ["DARM_rf", "CARM", "PRCL", "SRCL", "MICH"]:
    initial_gains[lsc_dof] = llo.get(f"{lsc_dof}_lock.gain")

In [ ]:
comparison_time = 1403893909 # Today (Jul 1st 24) last time IFO lost lock

#https://alog.ligo-la.caltech.edu/aLOG/index.php?callRep=71477
#comparison_time = to_gps('May 21 2024 23:00:00.000') # Valery's measurement is at 23:05 (time is UTC, so should match)
print(comparison_time)

max_time = 2*3600

#llodata = llo_data(chan_mapping=chns)
#llodata.reset_cache()
llodata = llo_data(chan_mapping='L1.yaml')

In [ ]:
# Prepare ndscope template file
myyaml = llodata.format_ndscope_yaml(comparison_time, max_time)
#print(myyaml)

In [ ]:
llodata.attach_ndscope_outputs(['2024_05_21-Parm.hdf5', '2024_06_01-PowerUp.hdf5'])

In [ ]:
Pin = llodata.get_timeseries(comparison_time, 'Pin', max_time, interpolate=True, frametype='L1_T', verbose=True, allow_tape=True)
G = llodata.get_timeseries(comparison_time, 'Guard', max_time, interpolate=True, verbose=True, allow_tape=True)

In [ ]:
data = time_dependant_sim(TS, llo, factory,
                          Pin, G, comparison_time, max_time, lock, 
                          cache_states=False, simDataKwargs={'enable': False})